In [1]:
# combining Costs file with Markets
# note that we will merge on the efficient measure name and the market at that is what 
# defines a measure (which is how we did the avoided costs (at the measure level)). However the market and 
# adoption model are at the condition level so after the merge we will have a lot of NAs 
# all the existing and baseline equipment will have NAs
# we will need to make sure they get pulled into the adoption model correctly
# the flags will need to force this
# this step is needed to get all the information the adoption model needs in one dataframe

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df = pd.read_pickle("020_input/measure_costs_benefits.plk")
df

,id,measure_name,sector,program,market,gas_utility,electric_utility,baseline_condition,efficient_condition,complementary_condition,...,TRC_BCR,SCT_cost,SCT_benefit,SCT_BCR,RIM_cost,RIM_benefit,RIM_BCR,PCT_cost,PCT_benefit,PCT_BCR
0,1,furnace_natural_gas_baseline_residential_to_fu...,residential,NLIRNC,NC,GasSellers,ElecCo,furnace_natural_gas_baseline_residential,furnace_natural_gas_efficient_residential,NaN,...,-5.117737,1540.0,-32783.870734,-21.288228,980.0,-7336.768525,-7.486498,700,-6845.335282,-9.779050
1,2,furnace_natural_gas_baseline_residential_to_fu...,residential,LIRNC,NC,GasSellers,ElecCo,furnace_natural_gas_baseline_residential,furnace_natural_gas_efficient_residential,NaN,...,-5.117737,1540.0,-32783.870734,-21.288228,980.0,-7336.768525,-7.486498,700,-6845.335282,-9.779050
2,3,furnace_natural_gas_baseline_residential_to_fu...,residential,NLIRNC,NC,GasSellers,ElecCo,furnace_natural_gas_baseline_residential,furnace_natural_gas_efficient_residential,NaN,...,-2.268975,1540.0,-14562.024080,-9.455860,980.0,-2949.674900,-3.009872,700,-2458.241657,-3.511774
3,4,furnace_natural_gas_baseline_residential_to_fu...,residential,LIRNC,NC,GasSellers,ElecCo,furnace_natural_gas_baseline_residential,furnace_natural_gas_efficient_residential,NaN,...,-2.268975,1540.0,-14562.024080,-9.455860,980.0,-2949.674900,-3.009872,700,-2458.241657,-3.511774
4,5,furnace_natural_gas_baseline_residential_to_fu...,residential,NLIRNC,NC,Methaners,ElecCo,furnace_natural_gas_baseline_residential,furnace_natural_gas_efficient_residential,NaN,...,-5.117737,1540.0,-32783.870734,-21.288228,980.0,-7336.768525,-7.486498,700,-6845.335282,-9.779050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,374,refrigerator_electricity_top10_residential_ref...,residential,NLIRRENO,RENO,Methaners,ElecCo,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,NaN,...,0.260472,1650.0,1368.972473,0.829680,1050.0,1013.325172,0.965072,750,1539.758415,2.053011
374,375,refrigerator_electricity_top10_residential_ref...,residential,NLIRRENO,RENO,Methaners,FreshTricity,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,NaN,...,0.260472,1650.0,1368.972473,0.829680,1050.0,1013.325172,0.965072,750,1539.758415,2.053011
375,376,refrigerator_electricity_top10_residential_ref...,residential,LIRRENO,RENO,GasSellers,ElecCo,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,NaN,...,0.260472,1650.0,1368.972473,0.829680,1050.0,1013.325172,0.965072,750,1539.758415,2.053011
376,377,refrigerator_electricity_top10_residential_ref...,residential,LIRRENO,RENO,Methaners,ElecCo,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,NaN,...,0.260472,1650.0,1368.972473,0.829680,1050.0,1013.325172,0.965072,750,1539.758415,2.053011


In [4]:
### making a tool###
# # make a list from df which is efficient_condition and measure_life_(yrs) and drop duplicates

#this will have to come from the workpapers 
df_eff_life = df[['efficient_condition', 'measure_life_(yrs)']].drop_duplicates()
df_base_life = df[['baseline_condition', 'measure_life_(yrs)']].drop_duplicates()
# now just stack these two dataframes on top of each other with the columns efficient_condition and baseline_condition renamed to condition
df_eff_life = df_eff_life.rename(columns={'efficient_condition': 'condition', 'measure_life_(yrs)': 'measure_life_(yrs)'})
df_base_life = df_base_life.rename(columns={'baseline_condition': 'condition', 'measure_life_(yrs)': 'measure_life_(yrs)'})

df_life = pd.concat([df_eff_life, df_base_life]).drop_duplicates().reset_index(drop=True)
df_life

,condition,measure_life_(yrs)
0,furnace_natural_gas_efficient_residential,10
1,cchp_electricity_efficient_residential,10
2,air_conditioner_electricity_efficient_residential,10
3,insulation_natural_gas_efficient_residential,30
4,HERS_electricity_efficient_residential,30
5,refrigerator_electricity_efficient_residential,10
6,refrigerator_electricity_top10_residential,10
7,furnace_natural_gas_baseline_residential,10
8,room_ac_electricity_baseline_residential,10
9,air_conditioner_electricity_baseline_residential,10


In [12]:
df_yr1 = pd.read_pickle("020_input/df_yr1_001.pkl")
# Apply filters
df_yr1 = df_yr1[
    (df_yr1['electric_utility'] == 'test_utility_1') &
    (df_yr1['gas_utility'] == 'test_utility_1') &
    (df_yr1['competition_group'] == 'refrigeration')
]
df_yr1

,condition_name,competition_group,subgroup,electric_utility,gas_utility,single_family_year_one,multi_family_year_one,single_family_li_year_one,multi_family_li_year_one
56,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,test_utility_1,12500.0,1500.0,6250.0,750.0
63,refrigerator_electricity_baseline_residential,refrigeration,full_size,test_utility_1,test_utility_1,12500.0,1500.0,6250.0,750.0
70,refrigerator_electricity_efficient_residential,refrigeration,full_size,test_utility_1,test_utility_1,12500.0,1500.0,6250.0,750.0
77,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_1,test_utility_1,12500.0,1500.0,6250.0,750.0


In [9]:
df_yr1 = pd.read_pickle("020_input/df_yr1_001.pkl")
# first need to change df_yr1 into long format with the current column names which are building types into rows and a new column for building type
df_yr1 = df_yr1.melt(id_vars=['condition_name', 'competition_group', 'subgroup', 'electric_utility', 'gas_utility'], var_name='building_type', value_name='initial_count')

df_yr1

,condition_name,competition_group,subgroup,electric_utility,gas_utility,building_type,initial_count
0,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,test_utility_1,single_family_year_one,10000.0
1,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,test_utility_2,single_family_year_one,20000.0
2,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_2,test_utility_2,single_family_year_one,55000.0
3,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,none,single_family_year_one,10000.0
4,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_2,none,single_family_year_one,11000.0
...,...,...,...,...,...,...,...
499,HERS_electric_baseline_residential,whole_home,none,test_utility_2,test_utility_2,multi_family_li_year_one,3500.0
500,HERS_electric_baseline_residential,whole_home,none,test_utility_1,none,multi_family_li_year_one,1250.0
501,HERS_electric_baseline_residential,whole_home,none,test_utility_2,none,multi_family_li_year_one,1250.0
502,HERS_electric_baseline_residential,whole_home,none,none,test_utility_1,multi_family_li_year_one,1250.0


In [ ]:
# use df_life to map measure life to df_yr1 based on efficient_condition
df_yr1 = pd.merge(df_yr1, df_life, left_on='condition_name', right_on='condition', how='left')

# now I am going to allocate the initial counts into the markets based on the stock and flow proportions from expert opinion
# there are three markets that I want to allocate into: RET_ER, REMAINING, and ROB 
# ROB is 1/EUL (measure_life (yrs)
# RET_ER is 1/3 of non ROB stock
# remaining is 2/3 of non ROB stock
df_yr1['ROB'] = df_yr1['initial_count'] / df_yr1['measure_life_(yrs)']  # assuming measure life of 15 years
####################################
######### Feature add ##############
####################################
# We need to also include Ret_add_on
# the adoption model is ready for this
df_yr1['RET_ADD_ON'] = (df_yr1['initial_count'] - df_yr1['ROB']) 
# RET_ADD_ON is all of the non ROB stock



df_yr1['RET_ER'] = (df_yr1['initial_count'] - df_yr1['ROB']) * (1/3)
df_yr1['REMAINING'] = (df_yr1['initial_count'] - df_yr1['ROB']) * (2/3)

df_yr1 = df_yr1.melt(id_vars=['condition_name', 'competition_group', 'subgroup', 'electric_utility', 'gas_utility', 'building_type', 'measure_life_(yrs)', 'initial_count', 'condition'], var_name='market', value_name='count')
#need to remove the text '_year_one' from the building_type column
df_yr1['building_type'] = df_yr1['building_type'].str.replace('_year_one', '')

In [6]:
# we need to add a check to make sure the Market Characterization sheet and the work papers have the same conditions in them
# we also need to add a check to make sure all existing and baseline versions are included as needed
# for now remove all nas on measure_life_(yrs)
df_yr1 = df_yr1.dropna(subset=['measure_life_(yrs)'])

In [7]:
# now we just merge df_yr1 and df on condition_name, competition_group, electric_utility, gas_utility, building_type, and market
df_yrs = pd.merge(df_yr1, df, left_on=['condition_name', 'electric_utility', 'gas_utility', 'building_type', 'market', 'measure_life_(yrs)'], right_on=['efficient_condition',  'electric_utility', 'gas_utility', 'building_type', 'market', 'measure_life_(yrs)'], how='left')
df_yrs 


,condition_name,competition_group,subgroup,electric_utility,gas_utility,building_type,measure_life_(yrs),initial_count,condition,market,...,TRC_BCR,SCT_cost,SCT_benefit,SCT_BCR,RIM_cost,RIM_benefit,RIM_BCR,PCT_cost,PCT_benefit,PCT_BCR
0,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,test_utility_1,single_family,10.0,10000.0,furnace_fuel_oil_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,test_utility_2,single_family,10.0,20000.0,furnace_fuel_oil_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_2,test_utility_2,single_family,10.0,55000.0,furnace_fuel_oil_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,none,single_family,10.0,10000.0,furnace_fuel_oil_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_2,none,single_family,10.0,11000.0,furnace_fuel_oil_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,insulation_natural_gas_efficient_residential,whole_home,none,test_utility_2,test_utility_2,multi_family_li,30.0,350.0,insulation_natural_gas_efficient_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1340,insulation_natural_gas_efficient_residential,whole_home,none,test_utility_1,none,multi_family_li,30.0,125.0,insulation_natural_gas_efficient_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1341,insulation_natural_gas_efficient_residential,whole_home,none,test_utility_2,none,multi_family_li,30.0,125.0,insulation_natural_gas_efficient_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,insulation_natural_gas_efficient_residential,whole_home,none,none,test_utility_1,multi_family_li,30.0,125.0,insulation_natural_gas_efficient_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Not removing NAs we need to make sure all the files have the shared column names and such

df_yrs.to_pickle("./020_output/df_yrs.pkl")
df_yrs.to_pickle("./030_input/df_yrs.pkl")